Nội dung bài viết nhằm mục đích dùng để so sánh hiệu suất giữa 3 thuật toán XGBoost, RandomForest và Decision Tree trong việc dự đoán kho hàng. Bài viết này là viết về thuật toán XGBoost. Dữ liệu ở đây là file product_train.csv. Nguồn: https://www.kaggle.com/datasets/jamsbrown/global-superstore-data-of-2016

# Đọc dữ liệu từ file

In [1]:
import pandas as pd
import numpy as np
#đọc dữ liệu từ file
df = pd.read_excel("E:\Jupyter NoteBook\superstore_2016.xlsx")
dataset=df.values

- Row ID: Đây là số thứ tự của hàng trong bảng dữ liệu.
- Order ID: Đây là mã định danh duy nhất cho mỗi đơn hàng.
- Order Date: Đây là ngày đặt hàng.
- Ship Date: Đây là ngày giao hàng.
- Ship Mode: Đây là phương thức vận chuyển hàng hóa (ví dụ: giao hàng tiêu chuẩn, giao hàng nhanh, v.v.).
- Customer ID: Đây là mã định danh duy nhất cho mỗi khách hàng.
- Customer Name: Đây là tên của khách hàng.
- Segment: Đây là phân đoạn thị trường hoặc nhóm khách hàng (ví dụ: khách hàng cá nhân, doanh nghiệp, v.v.).
- Postal Code: Đây là mã bưu chính hoặc mã vùng của địa chỉ khách hàng.
- City: Đây là tên thành phố nơi khách hàng đặt hàng hoặc nhận hàng.
- State: Đây là tên tiểu bang, tỉnh hoặc vùng lãnh thổ nơi khách hàng đặt hàng hoặc nhận hàng.
- Country: Đây là tên quốc gia nơi khách hàng đặt hàng hoặc nhận hàng.
- Region: Đây là khu vực hoặc vùng địa lý trong quốc gia nơi khách hàng đặt hàng hoặc nhận hàng.
- Market: Đây là thị trường hoặc lĩnh vực kinh doanh nơi sản phẩm được tiếp thị và bán.
- Product ID: Đây là số hoặc mã định danh duy nhất cho mỗi sản phẩm.
- Category: Đây là danh mục hoặc nhóm chính của sản phẩm, ví dụ: điện tử, đồ nội thất, đồ ăn, v.v.
- Sub-Category: Đây là phân loại con hoặc nhóm phụ thuộc vào danh mục chính, ví dụ: máy tính xách tay, bàn làm việc, thực phẩm đóng hộp, v.v.
- Product Name: Đây là tên hoặc mô tả chi tiết của sản phẩm cụ thể.
- Sales: Đây là số tiền thu được từ việc bán sản phẩm.
- Quantity: Đây là số lượng sản phẩm được bán.
- Discount: Đây là số tiền giảm giá được áp dụng cho sản phẩm.
- Profit: Đây là lợi nhuận thu được từ việc bán sản phẩm, được tính bằng Sales trừ đi các chi phí khác như chi phí mua hàng, chi
- Shipping Cost: Đây là chi phí vận chuyển hàng hóa từ nơi xuất phát đến nơi đích.
- Order Priority: Đây là mức độ ưu tiên của đơn hàng (ví dụ: cao, trung bình, thấp) đối với quá trình xử lý và giao hàng.

Các cột trong bảng dữ liệu này cung cấp thông tin về các đơn hàng, khách hàng, sản phẩm và các chỉ số kinh doanh như doanh thu, lợi nhuận, chi phí vận chuyển và mức độ ưu tiên của đơn hàng.

In [2]:
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.770,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.630,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.490,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.160,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.040,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2015-KE1642066-42174,2015-06-19,2015-06-19,Same Day,KE-1642066,Katrina Edelman,Corporate,NaN,Kure,...,OFF-FA-3072,Office Supplies,Fasteners,"Advantus Thumb Tacks, 12 Pack",65.100,5,0.0,4.5000,1.010,Medium
51286,34337,US-2014-ZD21925140-41765,2014-05-06,2014-05-10,Standard Class,ZD-219251408,Zuschuss Donatelli,Consumer,37421.0,Chattanooga,...,FUR-FU-4070,Furniture,Furnishings,"Eldon Image Series Desk Accessories, Burgundy",16.720,5,0.2,3.3440,1.930,High
51287,31315,CA-2012-ZD21925140-41147,2012-08-26,2012-08-31,Second Class,ZD-219251404,Zuschuss Donatelli,Consumer,94109.0,San Francisco,...,OFF-AR-5321,Office Supplies,Art,Newell 341,8.560,2,0.0,2.4824,1.580,High
51288,9596,MX-2013-RB1979518-41322,2013-02-17,2013-02-21,Standard Class,RB-1979518,Ross Baird,Home Office,NaN,Valinhos,...,OFF-BI-2919,Office Supplies,Binders,"Acco Index Tab, Economy",13.440,2,0.0,2.4000,1.003,Medium


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          51290 non-null  int64         
 1   Order ID        51290 non-null  object        
 2   Order Date      51290 non-null  datetime64[ns]
 3   Ship Date       51290 non-null  datetime64[ns]
 4   Ship Mode       51290 non-null  object        
 5   Customer ID     51290 non-null  object        
 6   Customer Name   51290 non-null  object        
 7   Segment         51290 non-null  object        
 8   Postal Code     9994 non-null   float64       
 9   City            51290 non-null  object        
 10  State           51290 non-null  object        
 11  Country         51290 non-null  object        
 12  Region          51290 non-null  object        
 13  Market          51290 non-null  object        
 14  Product ID      51290 non-null  object        
 15  Ca

# Xử lý dữ liệu

Chuyển đổi dữ liệu về dạng số với những cột có giá trị là object

In [4]:
from sklearn.preprocessing import LabelEncoder
# chuyển từ các cột dạng chữ sang dạng số
le = LabelEncoder()

is_Category = df.dtypes == object

category_column_list = df.columns[is_Category].tolist()

df[category_column_list] = df[category_column_list].apply(lambda col: le.fit_transform(col))

In [5]:
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,40098,2130,2014-11-11,2014-11-13,0,88,0,0,73120.0,2368,...,3738,2,13,2965,221.980,2,0.0,62.1544,40.770,1
1,26341,13126,2014-02-05,2014-02-07,2,8872,413,1,NaN,3532,...,286,0,5,2525,3709.395,9,0.1,-288.7650,923.630,0
2,25330,12852,2014-10-17,2014-10-18,0,3680,181,0,NaN,499,...,3681,2,13,2502,5175.171,9,0.1,919.9710,915.490,3
3,13524,7434,2014-01-28,2014-01-30,0,9575,425,2,NaN,376,...,3652,2,13,2414,2892.510,5,0.1,-96.5400,910.160,3
4,47221,22305,2014-11-05,2014-11-06,1,14167,633,0,NaN,859,...,3302,2,6,3158,2832.960,8,0.0,311.5200,903.040,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,14340,2015-06-19,2015-06-19,1,9354,428,1,NaN,1729,...,1861,1,8,239,65.100,5,0.0,4.5000,1.010,3
51286,34337,24879,2014-05-06,2014-05-10,3,17406,795,0,37421.0,674,...,509,0,9,1223,16.720,5,0.2,3.3440,1.930,1
51287,31315,1183,2012-08-26,2012-08-31,2,17405,795,0,94109.0,2856,...,1233,1,2,2468,8.560,2,0.0,2.4824,1.580,1
51288,9596,18433,2013-02-17,2013-02-21,3,13737,653,2,NaN,3360,...,1356,1,3,87,13.440,2,0.0,2.4000,1.003,3


Những giá trị NaN trong dữ liệu sẽ được chuyển thành giá trị trung vị của cột tương ứng

In [6]:
df.fillna(df.median(), inplace=True)

In [7]:
df=df.drop(['Row ID'],axis=1)
df=df.drop(['Order ID'],axis=1)
df=df.drop(['Order Date'],axis=1)
df=df.drop(['Ship Date'],axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ship Mode       51290 non-null  int32  
 1   Customer ID     51290 non-null  int32  
 2   Customer Name   51290 non-null  int32  
 3   Segment         51290 non-null  int32  
 4   Postal Code     51290 non-null  float64
 5   City            51290 non-null  int32  
 6   State           51290 non-null  int32  
 7   Country         51290 non-null  int32  
 8   Region          51290 non-null  int32  
 9   Market          51290 non-null  int32  
 10  Product ID      51290 non-null  int32  
 11  Category        51290 non-null  int32  
 12  Sub-Category    51290 non-null  int32  
 13  Product Name    51290 non-null  int32  
 14  Sales           51290 non-null  float64
 15  Quantity        51290 non-null  int64  
 16  Discount        51290 non-null  float64
 17  Profit          51290 non-null 

# Xây dựng mô hình XGBoost

Chia dữ liệu thành 2 tập train, test với tỉ lệ 8:2, ở đây giá trị cần dự đoán là mức độ ưu tiên trong việc xử lý đơn hàng trong cột Order Priority

In [9]:
# Chia dữ liệu train, test

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2, random_state=42)

x_train = train.drop(columns=['Order Priority'])
y_train = train['Order Priority']

x_test = test.drop(columns=['Order Priority'])
y_test = test['Order Priority']

Tạo mô hình và huấn luyện mô hình

In [10]:
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Huấn luyện mô hình
model_xgb = xgb.XGBClassifier()
model_xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

Đánh giá kết quả thu được

In [12]:
acc = model_xgb.score(x_test, y_test)
print(acc*100)

75.0048742444921
